In [1]:
import logging
from naslib.defaults.trainer import Trainer
from naslib.optimizers import DARTSOptimizer
from naslib.search_spaces import DartsSearchSpace
from naslib.utils import utils, setup_logger, get_config_from_args, set_seed, log_args
from naslib.search_spaces.core.graph import Graph, EdgeData
from naslib.search_spaces.core import primitives as ops
from torch import nn
from fvcore.common.config import CfgNode
from copy import deepcopy
from IPython.display import clear_output
import torch
from naslib.search_spaces.core.primitives import AbstractPrimitive

device: cuda:0
device: cpu
device: cuda:0
device: cuda:0
device: cuda:0
device: cuda:0


In [2]:
config = utils.get_config_from_args(config_type='nas')
config.optimizer = 'darts'
config.search.batch_size = 32
utils.set_seed(config.seed)
clear_output(wait=True)
utils.log_args(config)
config.search.epochs=10
config.search.learning_rate=0.001
logger = setup_logger(config.save + '/log.log')
logger.setLevel(logging.INFO)

In [3]:
config.seed

99

In [4]:
class Power(AbstractPrimitive):
    def __init__(self,power):
        super().__init__(locals())
        self.power=power
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.pow(x,self.power)
    def get_embedded_ops(self):
        return None

class Sqrt(AbstractPrimitive):
    def __init__(self,eps=1e-10):
        super().__init__(locals())
        self.eps = eps
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.pow(torch.maximum(x,torch.tensor(self.eps).repeat(x.shape).cuda()),.5)
    def get_embedded_ops(self):
        return None

class Sin(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.sin(x)
    def get_embedded_ops(self):
        return None
    
class Cos(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.cos(x)
    def get_embedded_ops(self):
        return None

class Abs_op(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.abs(x)
    def get_embedded_ops(self):
        return None

class Sign(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return x*-1
    def get_embedded_ops(self):
        return None

class Beta_mul(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return x * self.beta
    def get_embedded_ops(self):
        return None

class Beta_add(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return x + self.beta
    def get_embedded_ops(self):
        return None

class Log(AbstractPrimitive):
    def __init__(self,eps=1e-10):
        super().__init__(locals())
        self.eps = eps
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.log(torch.maximum(x,torch.tensor(self.eps).repeat(x.shape).cuda()))
    def get_embedded_ops(self):
        return None

class Exp(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        x=torch.exp(x)
        return x
    def get_embedded_ops(self):
        return None

class Sinh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.sinh(x)
    def get_embedded_ops(self):
        return None

class Cosh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.cosh(x)
    def get_embedded_ops(self):
        return None

class Tanh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.tanh(x)
    def get_embedded_ops(self):
        return None

class Asinh(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.asinh(x)
    def get_embedded_ops(self):
        return None
    
class Atan(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.atan(x)
    def get_embedded_ops(self):
        return None

class Sinc(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.sinc(x)
    def get_embedded_ops(self):
        return None

class Maximum0(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.maximum(x,torch.zeros(x.shape).cuda())
    def get_embedded_ops(self):
        return None
    
class Minimum0(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.minimum(x,torch.zeros(x.shape).cuda())
    def get_embedded_ops(self):
        return None
    
class Sigmoid(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.sigmoid(x)
    def get_embedded_ops(self):
        return None

class LogExp(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        x=torch.log(1+torch.exp(x))
        return x
    def get_embedded_ops(self):
        return None

class Exp2(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.exp(-torch.pow(x,2))
    def get_embedded_ops(self):
        return None

class Erf(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.erf(x)
    def get_embedded_ops(self):
        return None

class Beta(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return self.beta
    def get_embedded_ops(self):
        return None
    
class Add(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.add(x[0],x[1])
    def get_embedded_ops(self):
        return None

class Sub(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.sub(x[0],x[1])
    def get_embedded_ops(self):
        return None

class Mul(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.mul(x[0],x[1])
    def get_embedded_ops(self):
        return None

class Div(AbstractPrimitive):
    def __init__(self,eps=1e-10):
        super().__init__(locals())
        self.eps=eps
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.div(x[0],x[1] + self.eps)
    def get_embedded_ops(self):
        return None

class Maximum(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.maximum(x[0],x[1])
    def get_embedded_ops(self):
        return None
    
class Minimum(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.minimum(x[0],x[1])
    def get_embedded_ops(self):
        return None

class SigMul(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.mul(torch.sigmoid(x[0]),x[1])
    def get_embedded_ops(self):
        return None

class ExpBetaSub2(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.exp(-self.beta*torch.pow(torch.sub(x[0],x[1]),2))
    def get_embedded_ops(self):
        return None

class ExpBetaSubAbs(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.exp(-self.beta*torch.abs(torch.sub(x[0],x[1])))
    def get_embedded_ops(self):
        return None

class BetaMix(AbstractPrimitive):
    def __init__(self, channels):
        super().__init__(locals())
        self.beta = torch.nn.Parameter(torch.ones(channels))
    def forward(self,x, edge_data=None):
        x=x.clamp(-10,10)
        return torch.add(-self.beta*x[0],(1-self.beta)*x[1])
    def get_embedded_ops(self):
        return None

class Stack():
    def __init__(self):
        pass
    def __call__(self, tensors, edges_data=None):
        return torch.stack(tensors)

In [5]:
class SimpleSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'a_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        stages = ['a_stage_1', 'a_stage_2']

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2) # intermediate node
        activation_cell.add_node(3) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData())]) # mutable intermediate edge
        activation_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable output edge

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        for i, scope in zip(range(3, 5), stages):
            self.add_node(i, subgraph=deepcopy(activation_cell).set_scope(scope).set_input([i-1])) # activation cell i
            self.nodes[i]['subgraph'].name = scope
        self.add_node(5) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 5)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[4, 5].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in stages:
            self.update_edges(
                update_func=lambda edge: self._set_ops(edge),
                scope=scope,
                private_edge_data=True,
            )

    def _set_ops(self, edge):
        edge.data.set('op', [
            ops.Sequential(nn.ReLU()),
            ops.Sequential(nn.Hardswish()),
            ops.Sequential(nn.LeakyReLU()),
            ops.Sequential(nn.Identity())
        ])

In [6]:
class ComplexSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'u_stage_1',
        'u_stage_2',
        'b_stage_1'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        u_stages = ['u_stage_1', 'u_stage_2']
        
        # unary cell definition
        unary_cell = Graph()
        unary_cell.name = 'u_cell'
        unary_cell.add_node(1) # input node
        unary_cell.add_node(2) # intermediate node
        unary_cell.add_node(3) # output node
        unary_cell.add_edges_from([(1, 2, EdgeData())]) # mutable edge
        unary_cell.edges[1, 2].set('cell_name', 'u_cell')
        unary_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable edge
        
        # binary cell definition
        binary_cell = Graph()
        binary_cell.name = 'b_cell'
        binary_cell.add_node(1) # input node
        binary_cell.add_node(2) # input node
        binary_cell.add_node(3) # concatination node
        binary_cell.nodes[3]['comb_op'] = stack()
        binary_cell.add_node(4) # intermediate node
        binary_cell.add_node(5) # output node
        binary_cell.add_edges_from([(3, 4, EdgeData())]) # mutable edge
        binary_cell.edges[3, 4].set('cell_name', 'b_cell') 
        binary_cell.add_edges_from([(1, 3, EdgeData().finalize()),
                                    (2, 3, EdgeData().finalize()),
                                    (4, 5, EdgeData().finalize())]) # immutable edges
        
        # activation cell definition
        activation_cell = Graph()
        activation_cell.name = 'a_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2, subgraph=deepcopy(unary_cell).set_scope('u_stage_1').set_input([1])) # unary node
        activation_cell.nodes[2]['subgraph'].name = 'u_stage_1'
        activation_cell.add_node(3, subgraph=deepcopy(unary_cell).set_scope('u_stage_2').set_input([1])) # unary node
        activation_cell.nodes[3]['subgraph'].name = 'u_stage_2'
        activation_cell.add_node(4, subgraph=deepcopy(binary_cell).set_scope('b_stage_1').set_input([2, 3])) # binary node
        activation_cell.nodes[4]['subgraph'].name = 'b_stage_1'
        activation_cell.add_node(5) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize()), 
                                        (1, 3, EdgeData().finalize()),
                                        (2, 4, EdgeData().finalize()),
                                        (3, 4, EdgeData().finalize()), 
                                        (4, 5, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=deepcopy(activation_cell).set_input([2])) # activation cell
        self.nodes[3]['subgraph'].name = 'a_stage_1'
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in u_stages:
            self.update_edges(
                update_func=lambda edge: self._set_unary_ops(edge),
                scope=scope,
                private_edge_data=True,
            ) # set unary cell ops
        
        self.update_edges(
            update_func=lambda edge: self._set_binary_ops(edge),
            scope='b_stage_1',
            private_edge_data=True
        ) # set binary cell ops
        

    def _set_unary_ops(self, edge):
        edge.data.set('op', [ops.Identity(), ops.Zero(stride=1)]) 
        
        
    def _set_binary_ops(self, edge):
        edge.data.set('op', [Minimum(), Maximum()]) 

In [7]:
class RNNSearchSpace(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'u_stage_1',
        'u_stage_2',
        'u_stage_3',
        'u_stage_4',
        'b_stage_1',
        'b_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        u_stages = ['u_stage_1', 'u_stage_2', 'u_stage_3', 'u_stage_4']
        b_stages = ['b_stage_1', 'b_stage_2']
        
        # unary cell definition
        unary_cell = Graph()
        unary_cell.name = 'u_cell'
        unary_cell.add_node(1) # input node
        unary_cell.add_node(2) # intermediate node
        unary_cell.add_node(3) # output node
        unary_cell.add_edges_from([(1, 2, EdgeData())]) # mutable edge
        unary_cell.edges[1, 2].set('cell_name', 'u_cell')
        unary_cell.add_edges_from([(2, 3, EdgeData().finalize())]) # immutable edge
        
        # binary cell definition
        binary_cell = Graph()
        binary_cell.name = 'b_cell'
        binary_cell.add_node(1) # input node
        binary_cell.add_node(2) # input node
        binary_cell.add_node(3) # concatination node
        binary_cell.nodes[3]['comb_op'] = Stack()
        binary_cell.add_node(4) # intermediate node
        binary_cell.add_node(5) # output node
        binary_cell.add_edges_from([(3, 4, EdgeData())]) # mutable edge
        binary_cell.edges[3, 4].set('cell_name', 'b_cell') 
        binary_cell.add_edges_from([(1, 3, EdgeData().finalize()),
                                    (2, 3, EdgeData().finalize()),
                                    (4, 5, EdgeData().finalize())]) # immutable edges
        
        # activation cell definition
        activation_cell = Graph()
        activation_cell.name = 'a_cell'
        activation_cell.add_node(1) # input node
        activation_cell.add_node(2, subgraph=deepcopy(unary_cell).set_scope('u_stage_1').set_input([1])) # unary cell 1
        activation_cell.nodes[2]['subgraph'].name = 'u_stage_1'
        activation_cell.add_node(3, subgraph=deepcopy(unary_cell).set_scope('u_stage_2').set_input([1])) # unary cell 2
        activation_cell.nodes[3]['subgraph'].name = 'u_stage_2'
        activation_cell.add_node(4, subgraph=deepcopy(unary_cell).set_scope('u_stage_3').set_input([1])) # unary cell 3
        activation_cell.nodes[4]['subgraph'].name = 'u_stage_3'
        activation_cell.add_node(5, subgraph=deepcopy(binary_cell).set_scope('b_stage_1').set_input([2, 3])) # binary cell 1
        activation_cell.nodes[5]['subgraph'].name = 'b_stage_1'
        activation_cell.add_node(6, subgraph=deepcopy(unary_cell).set_scope('u_stage_4').set_input([5])) # unary cell 4
        activation_cell.nodes[6]['subgraph'].name = 'u_stage_4'
        activation_cell.add_node(7, subgraph=deepcopy(binary_cell).set_scope('b_stage_2').set_input([4, 6])) # binary cell 2
        activation_cell.nodes[7]['subgraph'].name = 'b_stage_2'
        activation_cell.add_node(8) # output node
        activation_cell.add_edges_from([(1, 2, EdgeData().finalize()), 
                                        (1, 3, EdgeData().finalize()),
                                        (1, 4, EdgeData().finalize()),
                                        (2, 5, EdgeData().finalize()),
                                        (3, 5, EdgeData().finalize()), 
                                        (4, 7, EdgeData().finalize()),
                                        (5, 6, EdgeData().finalize()),
                                        (6, 7, EdgeData().finalize()),
                                        (7, 8, EdgeData().finalize())])
        
        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3, subgraph=deepcopy(activation_cell).set_input([2])) # activation cell
        self.add_node(4) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 4)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[3, 4].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge
        
        for scope in u_stages:
            self.update_edges(
                update_func=lambda edge: self._set_unary_ops(edge),
                scope=scope,
                private_edge_data=True,
            ) # set unary cell ops
        
        for scope in b_stages:
            self.update_edges(
                update_func=lambda edge: self._set_binary_ops(edge),
                scope=scope,
                private_edge_data=True
            ) # set binary cell ops
        

    def _set_unary_ops(self, edge, channels=None):
        edge.data.set('op', [
            ops.Identity(), 
            ops.Zero(stride=1)
        ]) 
        
        
    def _set_binary_ops(self, edge, channels=None):
        edge.data.set('op', [
            Minimum(),
            Maximum(),
        ]) 

In [8]:
class Stack(AbstractPrimitive):
    def __init__(self):
        super().__init__(locals())

    def forward(self, x, edge_data=None):
        return torch.stack(x)

    def get_embedded_ops(self):
        return None


class UnStack(AbstractPrimitive):
    def __init__(self, dim=1):
        super().__init__(locals())
        self.dim = dim

    def forward(self, x, edge_data=None):
        return x[self.dim]

    def get_embedded_ops(self):
        return None


class RNNResNet20SearchSpace(Graph):
    """
    https://www.researchgate.net/figure/ResNet-20-architecture_fig3_351046093
    """

    OPTIMIZER_SCOPE = [
        f"activation_{i}" for i in range(1, 20)
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1)  # input node
        activation_cell.add_node(2)  # unary node / intermediate node
        activation_cell.add_node(3)  # unary node / intermediate node
        activation_cell.add_node(4)  # binary node / output node
        activation_cell.add_edges_from([(1, 2, EdgeData())])  # mutable intermediate edge
        activation_cell.add_edges_from([(1, 3, EdgeData())])  # mutable intermediate edge

        activation_cell.add_edges_from([(2, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.add_edges_from([(3, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.nodes[4]['comb_op'] = Stack()

        activation_cell.add_node(5)  # binary node / output node
        activation_cell.add_edges_from([(4, 5, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(6)
        activation_cell.add_edges_from([(5, 6, EdgeData().finalize())])  # unary node / intermediate node
        activation_cell.add_node(7)
        activation_cell.add_edges_from([(6, 7, EdgeData())])  # mutable intermediate edge
        activation_cell.add_node(8)
        activation_cell.add_edges_from([(1, 8, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(9)
        activation_cell.add_edges_from([(8, 9, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.add_edges_from([(7, 9, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.nodes[9]['comb_op'] = Stack()

        activation_cell.add_node(10)
        activation_cell.add_edges_from([(9, 10, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(11)
        activation_cell.add_edges_from([(10, 11, EdgeData().finalize())])  # mutable intermediate edge

        for tup in [(1, 2), (1, 3), (1, 8), (6, 7)]:  # unary operations
            activation_cell.edges[tup[0], tup[1]].set("op", [
                ops.Identity(),
                ops.Zero(stride=1),
                ops.Sequential(Power(2)),
                ops.Sequential(Power(3)),
                ops.Sequential(Sqrt()),
                ops.Sequential(Sin()),
                ops.Sequential(Cos()),
                ops.Sequential(Abs_op()),
                ops.Sequential(Sign()),
#                 ops.Sequential(Beta_mul(channels=32)),
#                 ops.Sequential(Beta_add(channels=32)),
                ops.Sequential(Log()),
                ops.Sequential(Exp()),
                ops.Sequential(Sinh()),
                ops.Sequential(Cosh()),
                ops.Sequential(Tanh()),
                ops.Sequential(Asinh()),
                ops.Sequential(Atan()),
                ops.Sequential(Sinc()),
                ops.Sequential(Maximum0()),
                ops.Sequential(Minimum0()),
                ops.Sequential(Sigmoid()),
                ops.Sequential(LogExp()),
                ops.Sequential(Exp2()),
                ops.Sequential(Erf()),
#                 ops.Sequential(Beta(channels=16)),    
            ])

        for tup in [(4, 5), (9, 10)]:
            activation_cell.edges[tup[0], tup[1]].set("op", [
                ops.Sequential(Add()),
                ops.Sequential(Sub()),
                ops.Sequential(Mul()),
                ops.Sequential(Div()),
                ops.Sequential(Maximum()),
                ops.Sequential(Minimum()),
                ops.Sequential(SigMul()),
#                 ops.Sequential(ExpBetaSub2(channels=32)),
#                 ops.Sequential(ExpBetaSubAbs(channels=32)),
#                 ops.Sequential(BetaMix(channels=32)),
            ])

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1)  # input
        self.add_node(2)  # intermediate
        self.add_node(3,
                      subgraph=activation_cell.copy().set_scope("activation_1").set_input([2]))  # activation cell 3
        #self.nodes[3]['subgraph'].name = "activation_1"

        self.add_node(4)
        self.add_node(5,
                      subgraph=activation_cell.copy().set_scope("activation_2").set_input([4]))  # activation cell 3
        #self.nodes[5]['subgraph'].name = "activation_2"

        self.add_node(6)
        self.add_node(7,
                      subgraph=activation_cell.copy().set_scope("activation_3").set_input([6]))  # activation cell 3
        #self.nodes[7]['subgraph'].name = "activation_3"

        self.add_edges_from([
            (1, 2, EdgeData()),
            (2, 3, EdgeData()),
            (3, 4, EdgeData()),
            (4, 5, EdgeData()),
            (5, 6, EdgeData()),
            (3, 6, EdgeData()),
            (6, 7, EdgeData())
        ])

        self.edges[1, 2].set('op',
                             ops.Sequential(nn.Conv2d(3, 16, 3, padding=1), ))  # convolutional edge
        self.edges[3, 4].set('op',
                             ops.Sequential(nn.Conv2d(16, 16, 3, padding=1), ))  # convolutional edge
        self.edges[5, 6].set('op',
                             ops.Sequential(nn.Conv2d(16, 16, 3, padding=1), ))  # convolutional edge

        conv_option = {
            "in_channels": 16,
            "out_channels": 16,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(7, 4, activation_cell, conv_option)
        self._create_base_block(11, 6, activation_cell, conv_option)

        conv_option_a = {
            "in_channels": 16,
            "out_channels": 32,
            "kernel_size": 3,
            "padding": 1,
            "stride": 2
        }
        conv_option_b = {
            "in_channels": 16,
            "out_channels": 32,
            "kernel_size": 1,
            "padding": 0,
            "stride": 2
        }
        self._create_reduction_block(15, 8, activation_cell, conv_option_a, conv_option_b)

        conv_option = {
            "in_channels": 32,
            "out_channels": 32,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(19, 10, activation_cell, conv_option)
        self._create_base_block(23, 12, activation_cell, conv_option)

        conv_option_a = {
            "in_channels": 32,
            "out_channels": 64,
            "kernel_size": 3,
            "padding": 1,
            "stride": 2
        }
        conv_option_b = {
            "in_channels": 32,
            "out_channels": 64,
            "kernel_size": 1,
            "padding": 0,
            "stride": 2
        }
        self._create_reduction_block(27, 14, activation_cell, conv_option_a, conv_option_b)

        conv_option = {
            "in_channels": 64,
            "out_channels": 64,
            "kernel_size": 3,
            "padding": 1
        }
        self._create_base_block(31, 16, activation_cell, conv_option)
        self._create_base_block(34, 18, activation_cell, conv_option)

        # add head
        self.add_node(39)
        self.add_edges_from([
            (38, 39, EdgeData())
        ])
        self.edges[38, 39].set('op',
                               ops.Sequential(
                                   nn.AvgPool2d(8),
                                   nn.Flatten(),
                                   nn.Linear(64, 10),
                                   nn.Softmax()
                               ))  # convolutional edge
        self.add_node(40)
        self.add_edges_from([
            (39, 40, EdgeData().finalize())
        ])

    def _create_base_block(self, start: int, stage: int, cell, conv_option: dict):
        self.add_node(start + 1)

        self.add_node(start + 2, subgraph=cell.copy().set_scope(f"activation_{stage}").set_input(
            [start + 1]))  # activation cell 3
        #self.nodes[start + 2]['subgraph'].name = f"activation_{stage}"

        self.add_node(start + 3)

        self.add_node(start + 4, subgraph=cell.copy().set_scope(f"activation_{stage + 1}").set_input(
            [start + 3]))  # activation cell 3
        #self.nodes[start + 4]['subgraph'].name = f"activation_{stage + 1}"

        self.add_edges_from([
            (start, start + 1, EdgeData()),
            (start, start + 3, EdgeData()),
            (start + 1, start + 2, EdgeData()),
            (start + 2, start + 3, EdgeData()),
            (start + 3, start + 4, EdgeData()),
        ])

        self.edges[start, start + 1].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option), ))  # convolutional edge
        self.edges[start + 2, start + 3].set('op',
                                             ops.Sequential(nn.Conv2d(**conv_option), ))  # convolutional edge

    def _create_reduction_block(self, start: int, stage: int, cell, conv_option_a: dict, conv_option_b: dict):
        self.add_node(start + 1)

        self.add_node(start + 2, subgraph=cell.copy().set_scope(f"activation_{stage}").set_input(
            [start + 1]))  # activation cell 3
        #self.nodes[start + 2]['subgraph'].name = f"activation_{stage}"

        self.add_node(start + 3)

        self.add_node(start + 4, subgraph=cell.copy().set_scope(f"activation_{stage + 1}").set_input(
            [start + 3]))  # activation cell 3
        #self.nodes[start + 4]['subgraph'].name = f"activation_{stage + 1}"

        self.add_edges_from([
            (start, start + 1, EdgeData()),
            (start, start + 3, EdgeData()),  # add conv
            (start + 1, start + 2, EdgeData()),
            (start + 2, start + 3, EdgeData()),
            (start + 3, start + 4, EdgeData()),
        ])

        self.edges[start, start + 1].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option_a), ))  # convolutional edge
        conv_option_a["in_channels"] = conv_option_a["out_channels"]
        conv_option_a["stride"] = 1

        self.edges[start, start + 3].set('op',
                                         ops.Sequential(nn.Conv2d(**conv_option_b), ))  # convolutional edge
        self.edges[start + 2, start + 3].set('op',
                                             ops.Sequential(nn.Conv2d(**conv_option_a), ))  # convolutional edge

In [7]:
class SimpleSearchSpaceComplexActivation(Graph):

    OPTIMIZER_SCOPE = [
        'a_stage_1',
        'a_stage_2'
    ]

    QUERYABLE = False

    def __init__(self):
        super().__init__()

        stages = ['a_stage_1', 'a_stage_2']

        # cell definition
        activation_cell = Graph()
        activation_cell.name = 'activation_cell'
        activation_cell.add_node(1)  # input node
        activation_cell.add_node(2)  # unary node / intermediate node
        activation_cell.add_node(3)  # unary node / intermediate node
        activation_cell.add_node(4)  # binary node / output node
        activation_cell.add_edges_from([(1, 2, EdgeData())])  # mutable intermediate edge
        activation_cell.add_edges_from([(1, 3, EdgeData())])  # mutable intermediate edge

        activation_cell.add_edges_from([(2, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.add_edges_from([(3, 4, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.nodes[4]['comb_op'] = Stack()

        activation_cell.add_node(5)  # binary node / output node
        activation_cell.add_edges_from([(4, 5, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(6)
        activation_cell.add_edges_from([(5, 6, EdgeData().finalize())])  # unary node / intermediate node
        activation_cell.add_node(7)
        activation_cell.add_edges_from([(6, 7, EdgeData())])  # mutable intermediate edge
        activation_cell.add_node(8)
        activation_cell.add_edges_from([(1, 8, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(9)
        activation_cell.add_edges_from([(8, 9, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.add_edges_from([(7, 9, EdgeData().finalize())])  # mutable intermediate edge
        activation_cell.nodes[9]['comb_op'] = Stack()

        activation_cell.add_node(10)
        activation_cell.add_edges_from([(9, 10, EdgeData())])  # mutable intermediate edge

        activation_cell.add_node(11)
        activation_cell.add_edges_from([(10, 11, EdgeData().finalize())])  # mutable intermediate edge

        for tup in [(1, 2), (1, 3), (1, 8), (6, 7)]:  # unary operations
            activation_cell.edges[tup[0], tup[1]].set("op", [i for i in [
                ops.Identity(),
                ops.Zero(stride=1),
                ops.Sequential(Power(2)),
                ops.Sequential(Power(3)),
                ops.Sequential(Sqrt()),
                ops.Sequential(Sin()),
                ops.Sequential(Cos()),
                ops.Sequential(Abs_op()),
                ops.Sequential(Sign()),
#                 ops.Sequential(Beta_mul(channels=32)),
#                 ops.Sequential(Beta_add(channels=32)),
                ops.Sequential(Log()),
                ops.Sequential(Exp()),
                ops.Sequential(Sinh()),
                ops.Sequential(Cosh()),
                ops.Sequential(Tanh()),
                ops.Sequential(Asinh()),
                ops.Sequential(Atan()),
                ops.Sequential(Sinc()),
                ops.Sequential(Maximum0()),
                ops.Sequential(Minimum0()),
                ops.Sequential(Sigmoid()),
                ops.Sequential(LogExp()),
                ops.Sequential(Exp2()),
                ops.Sequential(Erf()),
#                 ops.Sequential(Beta(channels=16)),    
            ][:]])

        for tup in [(4, 5), (9, 10)]:
            activation_cell.edges[tup[0], tup[1]].set("op", [i for i in [
                ops.Sequential(Add()),
                ops.Sequential(Sub()),
                ops.Sequential(Mul()),
                ops.Sequential(Div()),
                ops.Sequential(Maximum()),
                ops.Sequential(Minimum()),
                ops.Sequential(SigMul()),
#               ops.Sequential(ExpBetaSub2(channels=32)),
#               ops.Sequential(ExpBetaSubAbs(channels=32)),
#               ops.Sequential(BetaMix(channels=32)),
            ][:]])

        # macroarchitecture definition
        self.name = 'makrograph'
        self.add_node(1) # input node
        self.add_node(2) # intermediate node
        self.add_node(3,subgraph=activation_cell.copy().set_scope("a_stage_1").set_input([2]))  # activation cell 3
        self.add_node(5) # output node
        self.add_edges_from([(i, i+1, EdgeData()) for i in range(1, 5)])
        self.edges[1, 2].set('op',
            ops.Sequential(
                nn.Conv2d(3, 6, 5),
                nn.MaxPool2d(2),
                nn.Conv2d(6, 16, 5),
                nn.MaxPool2d(2),
                nn.Flatten()
            )) # convolutional edge
        self.edges[4, 5].set('op', 
            ops.Sequential(
                nn.Linear(400, 10), 
                nn.Softmax(dim=1)
            )) # linear edge

In [9]:
search_space = RNNResNet20SearchSpace()

In [8]:
search_space = SimpleSearchSpaceComplexActivation()

In [9]:
optimizer = DARTSOptimizer(config)
optimizer.adapt_search_space(search_space)

WARNING [07/13 11:31:17 nl.search_spaces.core.graph]: Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`
WARNING [07/13 11:31:17 nl.search_spaces.core.graph]: Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`


In [10]:
trainer = Trainer(optimizer, config)
trainer.search()

[07/13 11:31:18 nl.defaults.trainer]: param size = 0.006882MB
[07/13 11:31:18 nl.defaults.trainer]: Start training
Files already downloaded and verified
Files already downloaded and verified
[07/13 11:31:21 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
+0.001529, -0.000555, -0.000095, -0.000816, +0.001150, -0.000755, -0.000470, -0.002380, +0.000877, -0.000549, -0.000377, +0.000611, +0.001464, +0.000401, -0.001118, +0.000092, +0.001110, +0.001769, +0.001139, -0.001464, -0.002292, +0.000135, -0.000023, 17
-0.001366, +0.001118, -0.000766, +0.001588, -0.000654, +0.000865, +0.000068, +0.000319, -0.000595, +0.001064, +0.000822, +0.000472, +0.000078, -0.001169, +0.001535, +0.000990, -0.000028, +0.000868, -0.000395, +0.001506, -0.001716, +0.000950, -0.001593, 3
-0.000397, +0.000516, +0.001723, +0.000910, -0.000226, +0.000921, -0.000947, -0.000704, +0.001299, -0.000040, +0.001560, -0.000360, -0.000715, +0.000691, -0.001885, +0.000120, -0.000407, +0.001689, 

[07/13 11:31:22 nl.defaults.trainer]: cuda consumption
 |===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |     872 KB |   15155 KB |  565762 KB |  564890 KB |
|       from large pool |       0 KB |       0 KB |       0 KB |       0 KB |
|       from small pool |     872 KB |   15155 KB |  565762 KB |  564890 KB |
|---------------------------------------------------------------------------|
| Active memory         |     872 KB |   15155 KB |  565762 KB |  564890 KB |
|       

WARNING [07/13 11:33:25 nl.search_spaces.core.graph]: Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`
[07/13 11:33:25 nl.defaults.trainer]: Epoch 0 done. Train accuracy (top1, top5): 12.32889, 54.55778, Validation accuracy: 12.35341, 54.76190
[07/13 11:33:25 nl.optimizers.oneshot.darts.optimizer]: Arch weights (alphas, last column argmax): 
-0.000792, -0.002944, -0.005308, +0.022618, -0.002210, -0.003208, -0.003502, -0.003620, -0.004159, +0.002299, -0.002644, +0.021380, -0.001287, -0.002647, -0.002059, -0.002486, -0.001924, -0.002680, -0.001128, -0.004395, -0.005099, -0.002879, -0.002654, 3
+0.004731, -0.003555, -0.007343, +0.022213, +0.010245, +0.004874, +0.011434, +0.005140, -0.009271, -0.010344, +0.012323, +0.019612, +0.011629, +0.004214, +0.005464, +0.005098, +0.011347, +0.005835, -0.001042, +0.012860, +0.009688, +0.012322, +0.004433, 3
+0.002853, -0.006004, -0.022094, +0.050983, +0.003416, +0.004491, +0.002504, +0.0037

KeyboardInterrupt: 

In [ ]:
trainer.evaluate_oneshot()

In [16]:
optimizer.architectural_weights

ParameterList(
    (0): Parameter containing: [torch.cuda.FloatTensor of size 23 (GPU 0)]
    (1): Parameter containing: [torch.cuda.FloatTensor of size 23 (GPU 0)]
    (2): Parameter containing: [torch.cuda.FloatTensor of size 23 (GPU 0)]
    (3): Parameter containing: [torch.cuda.FloatTensor of size 7 (GPU 0)]
    (4): Parameter containing: [torch.cuda.FloatTensor of size 23 (GPU 0)]
    (5): Parameter containing: [torch.cuda.FloatTensor of size 7 (GPU 0)]
)